In [ ]:
# based on steps in https://www.tensorflow.org/lite/performance/post_training_integer_quant
# adding Keras layer by https://github.com/keras-team/keras/issues/2221

import tensorflow as tf
import numpy as np
import h5py
%matplotlib inline 
import matplotlib.pyplot as plt
from matplotlib.image import imread
import math

from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.python.keras.models import Sequential
import tensorflow.python.keras
# import tensorflow.contrib.keras as keras
from keras.utils import to_categorical

import os

(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

i = 8
plt.title('Value: ' + str(y_train[i]))
plt.imshow(x_train[i])
plt.show()
print(x_train[i][13][13])

In [ ]:
img_rows, img_cols = 28, 28
    
#Siapkan matrix training 
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)
# print the final input shape ready for training
print("Train matrix shape", x_train.shape)
print("Test matrix shape", x_test.shape)
# one-hot encoding using keras' numpy-related utilities
# one hot encoding = index neuron dengan nilai paling tinggi menjadi hasil klasifikasi
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", y_train.shape)

In [ ]:
import keras.backend as K
decimalpoint = 8
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), use_bias=False, activation=tf.nn.relu,input_shape=(28,28,1), name='layer_1'),
    # tf.keras.layers.Lambda(lambda x: K.round(x * (2 ** decimalpoint)) / (2 ** decimalpoint)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(16, (3,3), use_bias=False, activation=tf.nn.relu, name='layer_2'),
    # tf.keras.layers.Lambda(lambda x: K.round(x * (2 ** decimalpoint)) / (2 ** decimalpoint)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, use_bias=False, activation=tf.nn.relu, name='layer_3'),
    # tf.keras.layers.Lambda(lambda x: K.round(x * (2 ** decimalpoint)) / (2 ** decimalpoint)),
    tf.keras.layers.Dense(10, use_bias=False, activation=tf.nn.softmax, name='layer_Output'),
    # tf.keras.layers.Lambda(lambda x: K.round(x * (2 ** decimalpoint)) / (2 ** decimalpoint)),
])

# compile the models
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())

# training the model and saving metrics in history
history = model.fit(x_train, y_train,
          batch_size=128, epochs=5,
          verbose=1,
          validation_data=(x_test, y_test))

In [ ]:
# plotting the metrics
fig = plt.figure()
plt.subplot(2,1,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='lower right')

plt.subplot(2,1,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right')

plt.tight_layout()

fig

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
import pathlib
tflite_models_dir = pathlib.Path("../CNN-VLSI/mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)
tflite_model_file = tflite_models_dir/"mnist_model.tflite"
tflite_model_file.write_bytes(tflite_model)

In [ ]:
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
mnist_train, _ = tf.keras.datasets.mnist.load_data()
#mnist_train_reshaped = mnist_train[0].reshape(60000,784)
#images = tf.cast(mnist_train_reshaped.reshape(60000, 28, 28, 1), tf.float32) / 255.0
images = tf.cast(mnist_train[0].reshape(60000, 28, 28, 1), tf.float32) / 255.0
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)
def representative_data_gen():
  for input_value in mnist_ds.take(100):
    yield [input_value]

converter.representative_dataset = representative_data_gen
tflite_model_quant = converter.convert()
tflite_model_quant_file = tflite_models_dir/"mnist_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

In [ ]:
import matplotlib.pylab as pyplt

interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
interpreter.allocate_tensors()
test_image = np.expand_dims(test_images[0], axis=0).astype(np.float32)

input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]
interpreter.set_tensor(input_index, test_image)
interpreter.invoke()
predictions = interpreter.get_tensor(output_index)

pyplt.imshow(test_images[0])
template = "True:{true}, predicted:{predict}"
_ = pyplt.title(template.format(true= str(test_labels[0]), predict=str(np.argmax(predictions[0]))))
pyplt.grid(False)

In [ ]:
interpreter_quant = tf.lite.Interpreter(model_path=str(tflite_model_quant_file))
interpreter_quant.allocate_tensors()
input_index_quant = interpreter_quant.get_input_details()[0]["index"]
output_index_quant = interpreter_quant.get_output_details()[0]["index"]

input_index = interpreter_quant.get_input_details()[0]["index"]
output_index = interpreter_quant.get_output_details()[0]["index"]
interpreter_quant.set_tensor(input_index, test_image)
interpreter_quant.invoke()
predictions = interpreter_quant.get_tensor(output_index)

pyplt.imshow(test_images[0])
template = "True:{true}, predicted:{predict}"
_ = pyplt.title(template.format(true= str(test_labels[0]), predict=str(np.argmax(predictions[0]))))
pyplt.grid(False)

In [ ]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]
    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    for test_image in test_images:
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

    # Compare prediction results with ground truth labels to calculate accuracy.
    accurate_count = 0
    for index in range(len(prediction_digits)):
    if prediction_digits[index] == test_labels[index]:
        accurate_count += 1
    accuracy = accurate_count * 1.0 / len(prediction_digits)
    return accuracy
print("evaluation_moodel definition complete")

In [ ]:
print(evaluate_model(interpreter))
print(evaluate_model(interpreter_quant))